In [25]:
# %run plot_utils.ipynb
exec(open("plot_utils.py").read())

In [28]:
dir_name = '../output-tmp/test/result/'
# dir_name = '../output/22Apr-start-date-change/results-data/'
country_name = 'Italy'
load_data(dir_name+country_name+'.npz')
plot_all();


FileNotFoundError: [Errno 2] No such file or directory: '../output-tmp/test/result/Italy.npz'